['jh', 'hj l;']


In [5]:
import ephem

fnm = 'side/MPCORB.DAT'
with open(fnm, 'r+') as db:
    line = db.readline()
    while len(line) > 0:
        if line.startswith('---'):
            break
        line = db.readline()
    for i in xrange(1):
        line = db.readline()
        print line
        ephem.readdb(line)

00001    3.34  0.12 K172G 266.80164   72.90774   80.30986   10.59240  0.0756828  0.21403411   2.7678809  0 MPO389571  6651 113 1801-2016 0.60 M-v 30h MPCLINUX   0000      (1) Ceres              20161108



ValueError: Bogus: 00001    3.34  0.12 K172G 266.80164   72.90774   80.30986   10.59240  0.0756828  0.21403411   2.7678809  0 MPO389571  6651 113 1801-2016 0.60 M-v 30h MPCLINUX   0000      (1) Ceres              20161108


In [2]:
import de423
from jplephem import Ephemeris
#https://pypi.python.org/pypi/jplephem

e = Ephemeris(de423)
jed = 2444391.5  # 1980.06.01
print e.compute('mars', jed)

[[ -2.27891559e+08]
 [ -7.42380374e+07]
 [ -2.78883236e+07]
 [  7.63362697e+05]
 [ -1.62758469e+06]
 [ -7.67220749e+05]]


In [2]:
import ephem
import numpy as np
from utils import parse_coord

def dpm(year):
    ds = np.array([30]*12)
    ds[ :7:2] += 1
    ds[7: :2] += 1
    if year % 4 == 0:
        ds[1] = 29
    else:
        ds[1] = 28 
    return ds


def prev_date(date):
    ds = dpm(date[2])
    
    if date[0] == 1:
        if date[1] == 1:
            nd = [31, 12, date[2] - 1]
        else:
            nd = [ds[date[1]-2], date[1]-1, date[2]]
    else:
        nd = [date[0]-1, date[1], date[2]]
    return nd

def next_date(date):
    ds = dpm(date[2])
    
    if date[0] == ds[date[1]-1]:
        if date[1] == 12:
            nd = [1, 1, date[2] + 1]
        else:
            nd = [1, date[1]+1, date[2]]
    else:
        nd = [date[0]+1, date[1], date[2]]
    return nd

def ephsun(date):
    # Moscow coordinates
    (lat, lon) = (55.7522200, 37.6155600) 

    date2 = prev_date(date)
    
    time = '21:00' #
    sdate1 = str(date2[2]) + '/' + str(date2[1]) + '/' + str(date2[0]) + ' '
    sdate2 = str(date[2]) + '/' + str(date[1]) + '/' + str(date[0]) + ' '
    sun = ephem.Sun()
    #gatech = ephem.Observer()
    #gatech.lon, gatech.lat = str(lon), str(lat)
    
    gatech = sdate1 + time
    #gatech.date = sdate1 + time

    sun.compute(gatech)
    sun1 = [parse_coord(str(sun.a_ra), 'ra'), parse_coord(str(sun.a_dec), 'deg')]
    
    gatech = sdate2 + time #.date
    #gatech.date = sdate2 + time

    sun.compute(gatech)
    sun2 = [parse_coord(str(sun.a_ra), 'ra'), parse_coord(str(sun.a_dec), 'deg')]
    return sun1, sun2

def ephsun_iv(date1, date2):
    # Moscow coordinates
    (lat, lon) = (55.7522200, 37.6155600) 

    resra = []
    resdec = []
    curdate = prev_date(date1)
    
    time = '21:00' #
    sun = ephem.Sun()
    while curdate != date2:
        sdate = str(curdate[2]) + '/' + str(curdate[1]) + '/' + str(curdate[0]) + ' '
        gatech = sdate + time
        sun.compute(gatech)
        sun1 = [parse_coord(str(sun.a_ra), 'ra'), parse_coord(str(sun.a_dec), 'deg')]
        resra.append(sun1[0].f2hd())
        resdec.append(sun1[1].f2hd())
        curdate = next_date(curdate)
    return resra, resdec


date1 = [15, 11, 2015]
date2 = [1, 1, 2016]
print next_date(date1)
svra, svdec = ephsun_iv(date1, date2)    

def linpar(ra1, ra2):
    g = np.array([[ 1., 0.],
                  [-1., 1.]])
    t = np.array([ra1, ra2]).reshape(-1,1)
    return np.dot(g, t)

sun_a, sun_b = linpar(sv[12],sv[13])
f_sun = lambda x: sun_a + sun_b*x
t = np.linspace(0,1,24.)
print svdec

[16, 11, 2015]


NameError: name 'sv' is not defined

In [275]:
from jplephem.spk import SPK
from coord import coord
import numpy as np
import ephem

def bary2eq(pos):
    [a, b, c] = pos
    Dec = 180./np.pi*np.arctan(c / np.sqrt( a*a + b*b ))
    RA  = 180. + 180./np.pi*np.arctan(b / a)
    return RA / 15., Dec

def date2jed(date):
    st = ''
    for i in xrange(len(date)):
        x = date[i]
        st += str(x)
        if i < 2:
            st += '/'
        if i == 2:
            st += ' '
        if (i > 2) and (i < 5):
            if i != len(date) - 1:
                st += ':'
    djd = float(ephem.Date(st)) # DJD, dublin jd
    return st, djd + 2415020

def dmy2(date):
    return [date[2], date[1], date[0]]


def planetpos2(a, b, jed):
    # Sattelite-Saturn
    position1, _  = kernel[6, 699].compute_and_differentiate(jed) 
    # Satellite-Saturn + Saturn-Solar -> Satellite-Sollar
    position2, _  = kernel[0,   6].compute_and_differentiate(jed) 
    # Satellite-Sollar - Earth-Solar -> Satellite-Earth
    position3, _  = kernel[0,   3].compute_and_differentiate(jed) 
    # Satellite-Earth - Earth-Earth -> Fin!
    position4, _  = kernel[3, 399].compute_and_differentiate(jed) 

    pos = position1[:3] + position2 - position3 - position4 #
    return pos

def planetpos(barc, obj, jed):
    # Sattelite-Saturn
    position1, _  = kernel[barc, obj].compute_and_differentiate(jed) 
    # Satellite-Saturn + Saturn-Solar -> Satellite-Sollar
    position2, _  = kernel[0,   barc].compute_and_differentiate(jed) 
    # Satellite-Sollar - Earth-Solar -> Satellite-Earth
    position3, _  = kernel[0,   3].compute_and_differentiate(jed) 
    # Satellite-Earth - Earth-Earth -> Fin!
    position4, _  = kernel[3, 399].compute_and_differentiate(jed) 

    pos = position1[:3] + position2 - position3 - position4 #
    return pos

def ephplan(barc, obj, kernel, date):
    date1 = dmy2(date) + [21, 0, 0]
    st, jed = date2jed(date1)#2457061.5 # 8.02.2015 00:00
    pos = planetpos(barc, obj, jed)
    eqpos = bary2eq(pos)
    return eqpos

def ephplan_iv(barc, obj, kernel, date1, date2):
    # Moscow coordinates
    (lat, lon) = (55.7522200, 37.6155600) 

    resdec = []
    resra = []
    curdate = prev_date(date1)
    while curdate != date2:
        rd = ephplan(barc, obj, kernel, curdate)
        resra.append(rd[0])
        resdec.append(rd[1])
        curdate = next_date(curdate)
    return resra, resdec

def linpar(ra1, ra2):
    g = np.array([[ 1., 0.],
                  [-1., 1.]])
    t = np.array([ra1, ra2]).reshape(-1,1)
    par = np.dot(g, t)
    f = lambda x: par[0] + par[1]*x
    return f

dirnm = './saturn/'
fnm = 'sat382.bsp'

barc = 6
obj = 649

#date1 = [15, 11, 2015]
#date2 = [1, 1, 2016]
dm = dpm(year)
kernel = SPK.open(dirnm+fnm)
t = np.linspace(0,1,24.)

year = 2015

for month in range(11,13):
    fnm = './csv/' + str(month)+str(year)+'.csv'
    f = open(fnm, 'w+')
    f.write('Date,Time,Dist(RA),Dist(DEC),RA,DEC\n')

    date1 = [1, month, year]
    date2 = [dm[month-1], month, year]
    date2 = next_date(date2)
    svra, svdec = ephsun_iv(date1, date2)    
    pvra, pvdec = ephplan_iv(barc, obj, kernel, date1, date2)
    
    for i in xrange(len(pvra)-1):
        f_pl = linpar(pvra[i], pvra[i+1])
        f_pl_dec = linpar(pvdec[i], pvdec[i+1])
        f_sun = linpar(svra[i],svra[i+1])
        f_sun_dec = linpar(svdec[i],svdec[i+1])
        str_date = str(i+1) + '/' + str(month) + '/' + str(year)
        for k in xrange(t.size):
            tk = t[k]
            dra = f_sun(tk) - f_pl(tk)
            ddec = f_sun_dec(tk) - f_pl_dec(tk)
            #sdra = coord(0, 0, dra[0]*60*60, np.sign(dra), 'ra')
            #sddec = coord(0, 0, ddec[0]*60*60, np.sign(ddec), 'deg')
            sra_pl = coord(0, 0, f_pl(tk)[0]*60*60, np.sign(f_pl(tk)[0]), 'ra')
            sdec_pl = coord(0, 0, f_pl_dec(tk)[0]*60*60, np.sign(f_pl_dec(tk)), 'deg')                
            sra_sun = coord(0, 0, f_sun(tk)[0]*60*60, np.sign(f_sun(tk)[0]), 'ra')
            sdec_sun = coord(0, 0, f_sun_dec(tk)[0]*60*60, np.sign(f_sun_dec(tk)), 'deg')   
            #dra = sra_sun - sra_pl
            #ddec = sdec_sun - sdec_pl
            #dra = dra.f2hd()
            #dra = dra[0] % 12
            #ddec = ddec[0] % 180
            dra = dra[0]
            ddec = ddec[0]
            strtime = str(k) + ':' + '00'
            f.write(str_date + ',' + strtime + ',' + str(dra) + ',' + str(ddec) + ',' +  str(sra) + ',' +str(sdec)+'\n')
        f.write('\n')
    f.close()
    print "Month ready:", month, obj

print kernel





Month ready: 11 649
Month ready: 12 649
File type DAF/SPK and format LTL-IEEE with 18 segments:
2433282.50..2469807.50  Saturn Barycenter (6) -> Janus (610)
2433282.50..2469807.50  Saturn Barycenter (6) -> Epimetheus (611)
2433282.50..2469807.50  Saturn Barycenter (6) -> Helene (612)
2433282.50..2469807.50  Saturn Barycenter (6) -> Telesto (613)
2433282.50..2469807.50  Saturn Barycenter (6) -> Calypso (614)
2433282.50..2469807.50  Saturn Barycenter (6) -> Atlas (615)
2433282.50..2469807.50  Saturn Barycenter (6) -> Prometheus (616)
2433282.50..2469807.50  Saturn Barycenter (6) -> Pandora (617)
2433282.50..2469807.50  Saturn Barycenter (6) -> Methone (632)
2433282.50..2469807.50  Saturn Barycenter (6) -> Pallene (633)
2433282.50..2469807.50  Saturn Barycenter (6) -> Polydeuces (634)
2433282.50..2469807.50  Saturn Barycenter (6) -> Anthe (649)
2433282.50..2469807.50  Saturn Barycenter (6) -> Aegaeon (653)
2433282.50..2469807.50  Saturn Barycenter (6) -> Saturn (699)
2433282.50..2469807.5

In [4]:
from jplephem.spk import SPK
from coord import coord
import numpy as np
import ephem

dirnm = './'
fnm = 'codes_300ast_20100725.bsp'
fnm_planet = 'de432s.bsp'
year = 2015

dm = dpm(year)
kernel = SPK.open(dirnm+fnm)
kernel.planet = SPK
t = np.linspace(0,1,24.)

print kernel

File type DAF/SPK and format LTL-IEEE with 300 segments:
2378495.00..2524575.00  Sun (10) -> Ceres (2000001)
2378495.00..2524575.00  Sun (10) -> Pallas (2000002)
2378495.00..2524575.00  Sun (10) -> Vesta (2000004)
2378495.00..2524575.00  Sun (10) -> Unknown Target (2000029)
2378495.00..2524575.00  Sun (10) -> Unknown Target (2000016)
2378495.00..2524575.00  Sun (10) -> Unknown Target (2000015)
2378495.00..2524575.00  Sun (10) -> Unknown Target (2000010)
2378495.00..2524575.00  Sun (10) -> Unknown Target (2000007)
2378495.00..2524575.00  Sun (10) -> Unknown Target (2000006)
2378495.00..2524575.00  Sun (10) -> Unknown Target (2000003)
2378495.00..2524575.00  Sun (10) -> Unknown Target (2000345)
2378495.00..2524575.00  Sun (10) -> Unknown Target (2000012)
2378495.00..2524575.00  Sun (10) -> Unknown Target (2000442)
2378495.00..2524575.00  Sun (10) -> Unknown Target (2000027)
2378495.00..2524575.00  Sun (10) -> Unknown Target (2000287)
2378495.00..2524575.00  Sun (10) -> Unknown Target (20

In [10]:
import ephem
from coord import coord

import numpy as np
import sys, requests
import math
from scipy.optimize import brentq

def dpm(year):
    ds = np.array([30]*12)
    ds[ :7:2] += 1
    ds[7: :2] += 1
    if year % 4 == 0:
        ds[1] = 29
    else:
        ds[1] = 28 
    return ds

def next_date(date):
    ds = dpm(date[2])
    
    if date[0] == ds[date[1]-1]:
        if date[1] == 12:
            nd = [1, 1, date[2] + 1]
        else:
            nd = [1, date[1]+1, date[2]]
    else:
        nd = [date[0]+1, date[1], date[2]]
    return nd


def ephsun_year(year):
    # Moscow coordinates
    (lat, lon) = (55.7522200, 37.6155600) 

    time = '21:00' #
    days_per_month = dpm(year)
    gatech = str(year-1) + '/12/31 ' + time
    sun = ephem.Sun()
    sun.compute(gatech)
    rv = [[gatech, sun.a_ra, sun.a_dec]]
    
    for i, x in enumerate(days_per_month):
        for j in xrange(x):
            gatech = (str(year) + '/' + str(i+1) + '/' + str(j+1) + ' ' + time)
            sun.compute(gatech)
            rv.append([gatech, sun.a_ra, sun.a_dec])
    return rv

def mpc_data(r):
    r = r.split(',')
    params = {}
    #if num >= uplim:
    #    d = r[0].replace(' ', '+')
    #    params['designation'] = d
    #else:
    d = r[0].split(' ')
    params['number'] = d[0]
    params['json'] = 1
    
    req = requests.post(url, params, auth = ('mpc_ws', 'mpc!!ws'))
    y = req.text
    if y == '[]':
        return ''
    
    y = y.replace('{', '')
    y = y.replace('}', '')
    y = y.replace('[', '')
    y = y.replace(']', '')
    y = y.replace('"','')
    y = y.split(',')

    res = {}
    for x in y:
        z = x.split(':', 1)
        res[z[0]] = z[1]

    diam = res['diameter_neowise']
    if diam == 'null':
        diam = ''
    #name = res['designation']
    #while name.startswith('0'):
    #    name = name[1:]
    #if num < uplim:
    #name = '(' + name + ') '
    #tmp = res['name']
    #if tmp == 'null':
    #    tmp = res['']
    #name = name + res['name']
    return diam

year = 2016
DECLIM = 16.

days_per_month = dpm(year)
result = [[]]*len(days_per_month)

sun = ephem.Sun()
sun_eph = ephsun_year(year)
#print sun_eph

dbfnm = 'large.edb'#'MPCORB.DAT.edb'
url = 'http://mpc.cfa.harvard.edu/ws/search'

max_entry = 412#455144
save_int = 100
offset = 0

with open(dbfnm, 'r+') as f:
    r = '#'
    while r.startswith('#'):
        r = f.readline()
    for idx in xrange(max_entry):
        if idx > 0:
            r = f.readline()
        else:
            for p in xrange(offset):
                r = f.readline()
        yh = ephem.readdb(r)
        for i in xrange(len(sun_eph) - 1):
            if idx % save_int == 0:
                np.savez('asteroids', result)
            [gatech1, sun1_ra, sun1_dec] = sun_eph[i]
            yh.compute(gatech1)
            dif_ra1 = yh.a_ra - sun1_ra

            [gatech2, sun2_ra, sun2_dec] = sun_eph[i+1]
            yh.compute(gatech2)
            dif_ra2 = yh.a_ra - sun2_ra

            if (dif_ra1 * dif_ra2) < 0:
                a = ephem.date(gatech1)
                b = ephem.date(gatech2)
                def func(x):
                    sun.compute(x)
                    yh.compute(x)
                    dist = yh.a_ra - sun.a_ra
                    dist *= 180./np.pi
                    return dist
                t0 = brentq(func, a, b)
                sun.compute(t0)
                yh.compute(t0)
                
                if abs(sun.a_ra - yh.a_ra)* 180./np.pi > 1e-3:
                    continue
                
                date = gatech1.split(' ')[0]
                date = date.split('/')
                
                date = [int(date[2]), int(date[1]), int(date[0])]
                date = next_date(date)
                date = [str(x) for x in date]
                
                ind_month = int(date[1]) - 1
                if len(date[1]) == 1:
                    date[1] = ' ' + date[1]
                if len(date[0]) == 1:
                    date[0] = ' ' + date[0]
                date = date[0] + '/' + date[1] + '/' + date[2]
                time = coord(0, 0, (t0 - a)*24.*60.*60., 1, 'ra')
                diam = mpc_data(r)
                q = r.split(',')
                name = q[0]
                
                yh_ra = str(yh.a_ra)
                yh_dec = str(yh.a_dec)
                dist = ephem.degrees(yh.a_dec - sun.a_dec) 
                if abs(dist) * 180. / np.pi > DECLIM:
                    continue
                print (idx / float(max_entry))*100

                #print date, sun.a_ra, yh.a_ra, sun.a_ra - yh.a_ra

                dist = str(dist)
                
                output = date + ',' + str(time) + ',' + name + ',' + diam + ',' + dist + ',' + yh_ra + ',' + yh_dec
                result[ind_month] = result[ind_month] + [[t0, output]]
np.savez('asteroids', result)

dirnm = './csv/'

head = 'Date,Time,Name,Diameter,Distance,Ra,Dec\n'
for k in xrange(len(result)):
    f = open(dirnm + str(k+1)+'.csv', 'w+')
    res = sorted(result[k], key = lambda x : x[0])
    f.writelines(head)
    for p, g in res:
        f.writelines(g + '\n')
    f.close()
    print 'Month ', k+1, 'ready'
print 'Check it!'
            


#gatech = ephem.Observer()

0.0
0.728155339806
0.970873786408
1.21359223301
1.69902912621
2.42718446602
2.66990291262
2.91262135922
3.39805825243
3.64077669903
4.12621359223
4.36893203883
4.61165048544
4.85436893204
5.09708737864
5.33980582524
6.06796116505
6.31067961165
6.79611650485
7.28155339806
7.52427184466
8.00970873786
8.25242718447
8.49514563107
8.73786407767
8.98058252427
9.22330097087
9.46601941748
9.70873786408
9.95145631068
10.1941747573
10.4368932039
10.6796116505
10.9223300971
11.4077669903
11.6504854369
11.8932038835
12.1359223301
12.6213592233
12.8640776699
13.1067961165
13.3495145631
13.5922330097
14.0776699029
14.3203883495
14.5631067961
15.0485436893
15.5339805825
15.7766990291
16.0194174757
16.2621359223
16.5048543689
16.7475728155
16.9902912621
17.2330097087
17.4757281553
17.7184466019
17.9611650485
18.4466019417
18.6893203883
18.932038835
19.1747572816
19.4174757282
19.6601941748
19.9029126214
20.145631068
20.3883495146
20.6310679612
21.1165048544
21.359223301
21.6019417476
21.8446601942
22.

In [238]:
q  = sun.a_ra + 0.2
print q, ephem.degrees(q), q*180./np.pi
print idx

dirnm = './csv/'

head = 'Date,Time,Name,Diameter,Distance,Ra,Dec\n'
for k in xrange(len(result)):
    f = open(dirnm + str(k+1)+'.csv', 'w+')
    res = sorted(result[k], key = lambda x : x[0])
    f.writelines(head)
    for p, g in res:
        f.writelines(g + '\n')
    f.close()
    print 'Month ', k+1, 'ready'
print 'Check it!'

4.70546926818 269:36:12.7 269.603529695
11730
Month  1 ready
Month  2 ready
Month  3 ready
Month  4 ready
Month  5 ready
Month  6 ready
Month  7 ready
Month  8 ready
Month  9 ready
Month  10 ready
Month  11 ready
Month  12 ready
Check it!


In [5]:
y = []
y == []

True

In [151]:
import ephem
import numpy as np
import sys, requests
import math

def sun_eph():
    pass


dbfnm = 'MPCORB.DAT.edb'
url = 'http://mpc.cfa.harvard.edu/ws/search'

f = open(dbfnm, 'r+')

r = '#'
while r.startswith('#'):
    r = f.readline()
    #print 
    
for i in xrange(I):
    r = f.readline()
print r, 'kkk'
#while '  ' in r:
#    r = r.replace('  ', ' ')
#r = r.replace(' ', ',')
    


rv = mpc_data(r, I)
print rv

yh = ephem.readdb(r)
dt = '2015/09/28 22:10'
yh.compute(dt)
print("%s %s" % (yh.a_ra, yh.a_dec))
print("%s %s" % (ephem.constellation(yh), yh.mag))

s = ephem.Sun()
s.compute(dt)
print("%s %s" % (s.a_ra, s.a_dec))
print("%s %s" % (ephem.constellation(s), s.mag))

def estimate_diameter(obj):
    angle = obj.size / 3600. * np.pi/180.
    distance = obj.earth_distance * ephem.meters_per_au / 1000.
    return 2 * distance * np.sin(angle / 2.)

print estimate_diameter(yh)
print ephem.degrees(yh.a_dec - s.a_dec) 

2008 YA,e,31.20091,119.32641,201.3196,2.651856,0,0.8316239,237.1706,1/13/2016,2000.0,18,0.15
kkk
[u'']


IndexError: list index out of range

In [75]:
a = 'fff'
a.split?

In [ ]:
''.split

In [8]:
import ephem


fnm = 'MPCORB.DAT'

if False:
    f = open(fnm, 'r+')

    r = ''
    while not r.startswith('-'):
        r = f.readline()
        #print r
    for i in xrange(180000):
        r = f.readline()
    #while '  ' in r:
    #    r = r.replace('  ', ' ')
    #r = r.replace(' ', ',')
    r = r.split()
    for rr in r:
        print rr

    #nb = ephem.readdb('SRPTNGPALO Object,f|M|sp,'
    #                  +str(ephem.hours(ephem.degrees(str(coord.RA))))+
    #                  ','
    #                  +str(ephem.degrees(str(coord.DEC)))
    #                  +',0,
    #                  '+'2000')
    yh = ephem.readdb('C/2002 Y1 (Juels-Holvorcem),e,,,,,,,,,,,')
    yh.compute('2003/4/11')     
    print("%s %s" % (yh.ra, yh.dec))
    print("%s %s" % (ephem.constellation(yh), yh.mag))


import sys, requests
import math

url = 'http://mpc.cfa.harvard.edu/ws/search'

params = {}
#params['diameter_neowise_min'] = 20.
params['json'] = 1
#params['inclination_max']= 1.0

params['name'] = '1995%20UL40'
#params['number'] = 9160


r = requests.post(url, params, auth = ('mpc_ws', 'mpc!!ws'))

print len(r.text), r.text

tu = o
for obj in r:
    N = obj['ascending_node']
    i = obg['inclination']
    w = obj['argument_of_perihelion']
    a = obj['semimajor_axis']
    e = obj['eccentricity']
    M = obj['mean_anomaly']
    
    w1 = N + w # longitude of perihelion
    L = M + w1 # mean longitude
    q = a*(1-e) # perihelion distance
    Q =a*(1+e) # aphelion distance
    P = a**1.5 # orbitalperiod
    T = (M / 360.) / P # time of perihelion
    #v = 
    #e
    
    # eccentric anomaly
    E = M + e*(180/np.pi) * np.sin(M) * ( 1.0 + e * np.cos(M) )
    if e > 0.05:
        E0 = E
        E1 = E0*10.
        while abs(E0 - E1)>1e-5:
            E0 = E1
            E1 = E0 - ( E0 - e*(180/np.pi) * np.sin(E0) - M ) / ( 1 - e * np.cos(E0) )
        E = E1    
    xv = a * ( np.cos(E) - e )
    yv = a * ( np.sqrt(1.0 - e*e) * np.sin(E) )

    v = math.atan2( yv, xv )
    r = np.sqrt( xv*xv + yv*yv )
    
    # heliocentric
    xh = r * ( np.cos(N) * np.cos(v+w) - np.sin(N) * np.sin(v+w) * np.cos(i) )
    yh = r * ( np.sin(N) * np.cos(v+w) + np.cos(N) * np.sin(v+w) * np.cos(i) )
    zh = r * ( np.sin(v+w) * np.sin(i) )
        
    
########### Primary orbital elements
    #N = longitude of the ascending node
    #i = inclination to the ecliptic (plane of the Earth's orbit)
    #w = argument of perihelion
    #a = semi-major axis, or mean distance from Sun
    #e = eccentricity (0=circle, 0-1=ellipse, 1=parabola)
    #M = mean anomaly (0 at perihelion; increases uniformly with time)
    
    
######### Related orbital elements
    #w1 = N + w   = longitude of perihelion
    #L  = M + w1  = mean longitude
    #q  = a*(1-e) = perihelion distance
    #Q  = a*(1+e) = aphelion distance
    #P  = a ^ 1.5 = orbital period (years if a is in AU, astronomical units)
    #T  = Epoch_of_M - (M(deg)/360_deg) / P  = time of perihelion
    #v  = true anomaly (angle between position and perihelion)
    #E  = eccentric anomaly
    
    

2 []


NameError: name 'o' is not defined

In [27]:
{"absolute_magnitude":"4.13",
 "albedo_neowise":0.142,
 "albedo_neowise_unc":0.046,
 "albedo_updated_at":"2014-07-23T19:43:54Z",
 "aphelion_distance":"3.413",
 "apparition_end_date":null,
 "apparition_end_magn":null,
 "apparition_start_date":null,
 "apparition_start_magn":null,
 "arc_length":70495,
 "argument_of_perihelion":"309.92586",
 "ascending_node":"173.0968983",
 "b_minus_v":0.635,
 "b_minus_v_source":"M",
 "binary_object":null,
 "color_ignore":"???",
 "color_updated_at":"2014-07-23T19:44:17Z",
 "critical_list_numbered_object":false,
 "delta_v":14.2,"designation":"0000002",
 "diameter_neowise":544.0,
 "diameter_neowise_unc":60.71,
 "earth_moid":null,
 "eccentricity":"0.2313681",
 "epoch":"2014-05-23.0",
 "epoch_jd":"2456800.5",
 "first_observation_date_used":"1821-05-16.0",
 "first_opposition_used":"1821",
 "g_neowise":0.11,
 "greatest_elong":null,
 "greatest_elong_date":null,
 "greatest_elong_decl":null,
 "greatest_elong_magn":null,
 "h_neowise":4.06,
 "inclination":"34.84103",
 "jupiter_moid":null,
 "km_neo":false,
 "last_observation_date_used":"2014-05-20.0",
 "last_opposition_used":"2014",
 "lightcurve_notes":".",
 "lightcurve_quality":"3",
 "mars_moid":null,
 "mean_anomaly":"35.50312",
 "mean_daily_motion":"0.2135891",
 "mercury_moid":null,
 "name":"Pallas",
 "neo":false,
 "number":2,
 "observations":7714,
 "oppositions":103,
 "orbit_type":0,
 "orbit_uncertainty":"0",
 "orbit_updated_at":"2014-07-10T16:08:38Z",
 "p_vector_x":"-0.56149472",
 "p_vector_y":"0.81832779",
 "p_vector_z":"-0.1227327",
 "panstarrs_v_minus_gprime":-0.35,
 "panstarrs_v_minus_gprime_source":"P",
 "panstarrs_v_minus_iprime":0.32,
 "panstarrs_v_minus_iprime_source":"P",
 "panstarrs_v_minus_rprime":0.14,
 "panstarrs_v_minus_rprime_source":"P",
 "panstarrs_v_minus_uprime":0.0,
 "panstarrs_v_minus_wprime":-0.13,
 "panstarrs_v_minus_wprime_source":"P",
 "panstarrs_v_minus_yprime":0.32,
 "panstarrs_v_minus_yprime_source":"P",
 "panstarrs_v_minus_zprime":0.26,
 "panstarrs_v_minus_zprime_source":"P",
 "perihelion_date":"2013-12-07.77843",
 "perihelion_date_jd":"2456634.27843",
 "perihelion_distance":"2.1304346",
 "period":"4.61",
 "pha":false,
 "phase_slope":"0.11",
 "q_vector_x":"-0.82462646",
 "q_vector_y":"-0.54106127",
 "q_vector_z":"0.16505729",
 "rc_minus_ic":0.4,
 "rc_minus_ic_source":"D",
 "residual_rms":"0.58",
 "saturn_moid":null,
 "semimajor_axis":"2.771723",
 "spin_amplitude_flag":"-",
 "spin_max_amplitude":0.16,
 "spin_min_amplitude":0.03,
 "spin_period":7.81,
 "spin_period_description":null,
 "spin_period_flag":null,
 "spin_updated_at":"2014-10-27T13:51:07Z",
 "taxonomy_class":"B",
 "taxonomy_class_source":"B",
 "tisserand_jupiter":3.0,
 "u_minus_b":0.284,
 "u_minus_b_source":"M",
 "updated_at":"2014-10-27T13:51:07Z",
 "uranus_moid":null,
 "v_minus_gprime":-0.261,
 "v_minus_gprime_source":"m",
 "v_minus_iprime":0.321,
 "v_minus_iprime_source":"d",
 "v_minus_rc":0.361,
 "v_minus_rc_source":"T",
 "v_minus_rprime":0.157,
 "v_minus_rprime_source":"m",
 "v_minus_uprime":0.0,
 "v_minus_wprime":0.0,
 "v_minus_yprime":0.0,
 "v_minus_zprime":0.435,
 "v_minus_zprime_source":"d",
 "venus_moid":null}

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(pv, label = 'Saturn')
plt.plot(sv, label = 'Sun')
plt.legend()
plt.grid()
plt.xlim(14.5, 16.5)
plt.show()
plt.plot(pv, label = 'Saturn')
plt.plot(sv, label = 'Sun')
plt.legend()
plt.grid()
plt.xlim(12, 20)
plt.show()


In [202]:
print dm

[31 28 31 30 31 30 31 31 30 31 30 31]


In [122]:
from compute import *
from utils import parse_date, parse_coord
import time

#from vdb import process_vdb, handle_date

#day = int(time.strftime("%d"))
month = 1#int(time.strftime("%m"))
year = int('20' + time.strftime('%y'))
dm = dpm(year)
vphi = 20
phi= coord(vphi,0,0,1,'deg')

for month in range(1,13):
    days = range(1, dm[month-1]+1)
    fnm = './csv/' + str(month)+str(year)+'.csv'
    f = open(fnm, 'w+')
    f.write('Date,Time,Dist,RA,DEC\n')


    for i in xrange(len(days)):
        curdate = [days[i], month, year]
        [[ra1, dec1], [ra2, dec2]] = ephsun(curdate)

        if dec1.f2s() < dec2.f2s():
            c = dec1
            d = dec2
        else:
            c = dec2
            d = dec1

        ra_interest = [ra1.f2hd(), ra2.f2hd()]
        dec_interest = [(c - phi).f2hd(), (d + phi).f2hd()]

        #fields = ['RA', 'DEC', 'flux(V)']
        customSimbad = Simbad()#columns = fields)

        query = "cat in ('" + cat + "')" + ' & '
        query += 'rah > '+ str(ra_interest[0])  + ' & ' + 'rah < ' + str(ra_interest[1])+ ' & '
        query += 'dec > '+ str(dec_interest[0]) + ' & ' + 'dec < ' + str(dec_interest[1])

        if cat == 'hip':
            customSimbad.add_votable_fields('flux(V)', 'id(hip)')
        elif cat == 'ngc':
            customSimbad.add_votable_fields('id(m)')
        customSimbad.ROW_LIMIT = -1


        result = customSimbad.query_criteria(query)


        if result is not None:
            ll = len(result)
            result.sort('RA')
        else:
            continue

        for i in xrange(ll):
            row = result[i]
            [ra, dec] = [row['RA'], row['DEC']]
            if cat == 'hip':
                vmag = row['FLUX_V']
            ra = parse_coord( ra, 'ra', ' ')
            dec = parse_coord( dec, 'deg', ' ')
            try:
                t = comp_time(ra1, ra2, ra)
            except:
                print ra1, ra2, ra, row['MAIN_ID']
                continue
                
            tmp = str(curdate[0])
            if len(tmp) <  2:
                tmp = ' ' + tmp
            date = '' + tmp + '/'
            tmp = str(curdate[1])
            if len(tmp) < 2:
                tmp = ' ' + tmp
            date = date + tmp + '/' + str(curdate[2])
            dist = comp_dist(curdate[0], curdate[1], t, dec)

            hdist = abs(dist.f2s() / 3600.)
            if (cat == 'hip')and False:
                if (str(vmag)!='--'):
                    if (vmag > 3):
                        if (vmag <= 4):
                            if hdist > 15:
                                continue
                        elif (vmag <= 5):
                            if hdist > 14:
                                continue
                        elif (vmag <= 6):
                            if hdist > 13:
                                continue
                        elif (vmag <= 7):
                            if hdist > 12:
                                continue
                        elif (vmag <= 8):
                            if hdist > 8:
                                continue
                        elif (vmag <= 9):
                            if hdist > 4:
                                continue
                        else: #(vmag <= 10):
                            if hdist > 2:
                                continue
                        #else:
                            #print "Strange! " + row['MAIN_ID'], vmag, hdist
                            #continue
                            
            printer = '' + date + ',' + t.__str__(rpar = 0) 

            stri = [row['MAIN_ID'], str(dist)]
            if cat == 'hip':
                alt_names = Simbad.query_objectids(row['MAIN_ID'])
                stnms = ''
                dl = ''
                hd = ''
                for an in alt_names:
                    if an['ID'].startswith('HD'):
                        if len(hd) > 0:
                            hd = hd + ' / '
                        hd = hd + an['ID']
                    if an['ID'].startswith('* '):
                        if len(dl) > 0:
                            dl = dl + ' / '
                        dl = dl + an['ID'].replace("*", "")
                    if an['ID'].startswith('NAME '):
                        if len(stnms) > 0:
                            stnms = stnms + ' / '
                        tmp = an['ID'].replace("NAME", '')
                        stnms = stnms + tmp
                stri = [row['ID_hip'], hd, dl, stnms, str(dist), str(vmag)]
            elif cat == 'ngc':
                mes = row['ID_m']
                if (len(mes) == 0) and (hdist > 15):
                    continue
                alt_names = Simbad.query_objectids(row['MAIN_ID'])
                stnms = ''
                ngc = ''
                for an in alt_names:
                    if an['ID'].startswith('NGC '):
                        if len(ngc) > 0:
                            ngc = ngc + ' / '
                        ngc = ngc + an['ID']
                    if an['ID'].startswith('NAME '):
                        if len(stnms) > 0:
                            stnms = stnms + ' / '
                        tmp = an['ID'].replace("NAME", '')
                        stnms = stnms + tmp    
                stri = [ngc, mes, stnms, str(dist)]
            stri += [str(ra), str(dec)]

            for x in stri:
                printer = printer + ',' + x
            f.writelines(printer+'\n')
            #print printer
        f.write('\n')
    f.close()
    print "Month", month, "is ready!"
print "Check it!"

16:16:46.49
